# Solana Network Analysis

This notebook analyzes various aspects of the Solana network, addressing the following questions:

1. What is the current validator count?
2. What is the average block time for 25 blocks from a recent epoch?
3. What is the average block time for 25 blocks from an epoch in 2022?
4. What is the current real yield earned by a staker?

We'll use the Solana RPC API to fetch this data and perform our calculations.

In [1]:
import time
from solders.pubkey import Pubkey
from solders.signature import Signature
from solana.rpc.async_api import AsyncClient
from solana.rpc.commitment import Confirmed
from solana.rpc.core import RPCException
import asyncio
from datetime import datetime

# Define the API endpoint as a constant
API_ENDPOINT = "https://greatest-autumn-silence.solana-mainnet.quiknode.pro/44f458e9cc436b5174516aaa2830e1f02b525aa2"

## 1. Current Validator Count

It uses the `get_vote_accounts()` method from the Solana RPC API to retrieve this information.

The validator count is broken down into three categories:
1. Current: The number of validators currently participating in consensus.
2. Delinquent: The number of validators that are not actively participating (e.g., offline or not up-to-date).
3. Total: The sum of current and delinquent validators.



In [2]:
async def get_validator_count():
    async with AsyncClient(API_ENDPOINT) as client:
        response = await client.get_vote_accounts()
        return {
            "current": len(response.value.current),
            "delinquent": len(response.value.delinquent),
            "total": len(response.value.current) + len(response.value.delinquent)
        }

# Fetch and print validator count
validator_count = await get_validator_count()
print(f"Validator count: {validator_count}")

Validator count: {'current': 1378, 'delinquent': 50, 'total': 1428}


## 2. Average Block Time for Recent Blocks

The following function calculates the average block time for a specified number of recent blocks.
It handles potential RPC exceptions and accounts for skipped blocks in the calculation.

Here's how it works:

1. If no start slot is provided, it fetches the latest blockhash to get the most recent block height.
2. It then iterates backwards from this slot, fetching block times for each slot.
3. If a block is missing or there's an RPC exception:
   - It increments a 'failed_attempts' counter
   - Moves to the next slot
4. It continues until it has collected enough valid block times (num_blocks + 1 + failed_attempts).
5. The function then sorts the collected slots and calculates time differences only for consecutive blocks.
6. Finally, it computes the average of these time differences to get the average block time.

This approach ensures that:
- Skipped blocks don't affect the average calculation
- We have enough data points even if some blocks are missing or inaccessible
- RPC exceptions are handled gracefully without interrupting the overall process




In [3]:
async def get_blocks_and_average_time(num_blocks, start_slot=None):
    async with AsyncClient(API_ENDPOINT) as client:
        if start_slot is None:
            recent_blockhash = await client.get_latest_blockhash()
            start_slot = recent_blockhash.value.last_valid_block_height
        
        slot_to_timestamp = {}
        slot = start_slot
        failed_attempts = 0

        while len(slot_to_timestamp) < num_blocks + 1 + failed_attempts:
            try:
                timestampResponse = await client.get_block_time(slot)
                if timestampResponse.value is not None:
                    slot_to_timestamp[slot] = timestampResponse.value
                else:
                    failed_attempts += 1
                slot -= 1
            except RPCException as e:
                if "skipped, or missing in long-term storage" in str(e):
                    failed_attempts += 1
                    slot -= 1  # Try the next slot
                else:
                    raise  # Re-raise if it's a different kind of error

        if len(slot_to_timestamp) < num_blocks + 1:
            raise ValueError(f"Could not fetch enough valid blocks. Only got {len(slot_to_timestamp)} slots.")

        # Sort slots in descending order (newest to oldest)
        sorted_slots = sorted(slot_to_timestamp.keys(), reverse=True)

        # Calculate time differences, skipping when blocks are missing
        time_diffs = []
        for i in range(len(sorted_slots) - 1):
            if sorted_slots[i] - sorted_slots[i+1] == 1:  # Check if blocks are consecutive
                time_diff = slot_to_timestamp[sorted_slots[i]] - slot_to_timestamp[sorted_slots[i+1]]
                time_diffs.append(time_diff)
            
            if len(time_diffs) == num_blocks:
                break

        if len(time_diffs) < num_blocks:
            raise ValueError(f"Could not calculate enough valid time differences. Only got {len(time_diffs)} differences.")

        avg_block_time = sum(time_diffs) / len(time_diffs)
        
        return avg_block_time

# Calculate average block time for recent epoch
try:
    recent_avg_block_time = await get_blocks_and_average_time(25)
    print(f"Average block time (recent epoch): {recent_avg_block_time:.2f} seconds")
except Exception as e:
    print(f"Error calculating recent average block time: {e}")

# Calculate average block time for epoch in 2022
slot_2022 = 120000000  # This is an approximate slot number for an epoch in 2022
try:
    avg_block_time_2022 = await get_blocks_and_average_time(25, slot_2022)
    print(f"Average block time (2022 epoch): {avg_block_time_2022:.2f} seconds")
except Exception as e:
    print(f"Error calculating 2022 average block time: {e}")

Average block time (recent epoch): 0.44 seconds
Average block time (2022 epoch): 0.56 seconds


# Calculating Nominal and Real Yield

In the following cells, we'll calculate the nominal yield and then convert it to real yield. We'll also calculate historical inflation rates and real yields for comparison.

## Nominal Yield Calculation

The `estimate_real_yield` function calculates the nominal yield using the following formula:

nominal_yield = inflation_rate * (target_slot_time / actual_slot_time) / staked_ratio

Where:
- inflation_rate: The current inflation rate of Solana
- target_slot_time: Solana's target slot time (0.4 seconds)
- actual_slot_time: The actual average slot time we calculated earlier
- staked_ratio: The proportion of total supply that is staked

## Real Yield Calculation

The real yield is then calculated by adjusting the nominal yield for inflation:

real_yield = (1 + nominal_yield) / (1 + inflation_rate) - 1

This gives us the yield in real terms, accounting for the effects of inflation.

## Historical Calculations

We'll also calculate the historical inflation rate and real yield for 2022:

1. We use the disinflation rate to estimate the inflation rate in 2022.
2. We assume a staked ratio of 60% for 2022.
3. We use the block time from 2022 that we calculated earlier.

These calculations allow us to compare current yields with historical yields and understand how they've changed over time.


In [23]:
def estimate_real_yield(inflation_rate, staked_ratio, avg_slot_time):
    est = 0.4  # Solana's target slot time
    nominal_yield = inflation_rate * est / avg_slot_time / staked_ratio
    real_yield = 1 + nominal_yield / (1 + inflation_rate) - 1
    return real_yield

In [26]:
async with AsyncClient(API_ENDPOINT) as client:
    # Get staked and total supply
    supply_response = await client.get_supply()
    total_circulating_supply = supply_response.value.circulating/1000000000

    # Get inflation rate
    inflation_rate_response = await client.get_inflation_rate()
    inflation_per_annum = inflation_rate_response.value.total

staked_supply = 306.9 * 10**6
staked_ratio = staked_supply / total_circulating_supply

# Get average slot time over last 25 blocks
stAvg = recent_avg_block_time 
# Calculate real yield
real_yield = estimate_real_yield(inflation_per_annum, staked_ratio, stAvg)

print(f"Real Yield: {real_yield*100:.4f}%")


Real Yield: 6.6391%


In [38]:
async with AsyncClient(API_ENDPOINT) as client:
    # Get staked and total supply
    supply_response = await client.get_inflation_governor()
    disinflation_rate = supply_response.value.taper
    
    block_time_2022 = await client.get_block_time(120000000)

today = datetime.now()
# calculate the inflation_rate_per_annum_2022 basis disinflation_rate and time difference between today and 2022
time_diff = today - datetime(2022, 1, 1)
inflation_rate_per_annum_2022 = inflation_per_annum / ((1-disinflation_rate) ** (time_diff.days / 365))

# assuming the staked ratio was about 60% in 2022
staked_ratio_2022 = 0.6
avg_slot_time_2022 = avg_block_time_2022

historical_real_yield = estimate_real_yield(inflation_rate_per_annum_2022, staked_ratio_2022, avg_slot_time_2022)
print(f"Historical Real Yield 2022: {historical_real_yield*100:.4f}%")

Historical Real Yield 2022: 8.6194%


## Conclusion

This notebook has addressed the following questions about the Solana network:

1. We've fetched the current validator count, including active and delinquent validators.
2. We've calculated the average block time for 25 blocks from a recent epoch.
3. We've calculated the average block time for 25 blocks from an epoch in 2022.
4. We've estimated the current real yield earned by a native staker.

Note that the real yield calculation is a simplified estimate and doesn't include factors such as validator commission fees or potential slashing events. For a more accurate historical real yield, you would need to incorporate historical data for inflation rates, staking ratios, and actual validator performance over time.

This analysis provides insights into the current state of the Solana network and how some key metrics have changed over time. It can be useful for understanding network performance and potential staking returns.